<a href="https://colab.research.google.com/github/luthierman/quantum-research/blob/main/blackjack_quantum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.3.1
!pip install tensorflow-quantum
!pip install gym

     |████████████████████████████████| 320.4MB 48kB/s 
     |████████████████████████████████| 460kB 39.3MB/s 
     |████████████████████████████████| 20.1MB 1.3MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1


     |████████████████████████████████| 5.9MB 4.1MB/s 
     |████████████████████████████████| 1.6MB 43.8MB/s 
     |████████████████████████████████| 5.6MB 50.9MB/s 
  Found existing installation: sympy 1.7.1
    Uninstalling sympy-1.7.1:
      Successfully uninstalled sympy-1.7.1


In [ ]:
import os.path
from os import path
def make_path(p, d):
  print("Checking if {} exists...".format(p+d))
  if path.exists(p+d) == False:
    print("making... new directory")
    os.mkdir(p+str(d))
  print("finished!")
  print(p+str(d))
  return p+str(d)

In [ ]:
import gym
import numpy as np
import random
import matplotlib.pyplot as plt
import math
from collections import deque
import tensorflow as tf
import datetime
import time
import tensorflow_quantum as tfq
import cirq 
import sympy
import gym
import numpy as np
import random
import matplotlib.pyplot as plt
import math
from collections import deque
import tensorflow as tf
import datetime
import time
import tensorflow_quantum as tfq
import cirq 
import sympy
class QDQN_alt(object):
    def __init__(self, action_space, state_space, no_qubits=4) -> None:
        super().__init__()
        self.action_space = action_space
        self.state_space = state_space
        self.no_qubits = no_qubits
        self.qubits = [cirq.GridQubit(0, i) for i in range(no_qubits)]
        self.q_network = self.make_func_approx()
        self.learning_rate = 0.01
        self.opt = tf.keras.optimizers.Adam(lr=self.learning_rate)
        self.buff = 10000
        self.batch = 32   
        self.states = np.zeros((self.buff, self.state_space))
        self.actions = np.zeros((self.buff, 1))
        self.rewards = np.zeros((self.buff, 1))
        self.dones = np.zeros((self.buff, 1))
        self.next_states = np.zeros((self.buff, self.state_space))
        # Q Learning
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_decay = 0.9
        self.epsilon_min = 0.01
        self.counter = 0
        self.date = datetime.date.today()
        self.model_name = "QDQN-{date}_qbits{q}_ADAM_lr{lr}_bs{bs}_g{g}_eps{ep}_epsmin{epmin}_epsd{epd}".format(
            date=self.date,
            q=self.no_qubits,g=self.gamma, bs=self.batch,
            lr=self.learning_rate,
            ep=self.epsilon,
            epmin=self.epsilon_min,
            epd=self.epsilon_decay)
        self.msbe =None
    def make_func_approx(self):
        readout_operators = [cirq.Z(self.qubits[i]) for i in range(2,4)]
        inputs = tf.keras.Input(shape=(), dtype=tf.dtypes.string)
        diff = tfq.differentiators.ParameterShift()
        init = tf.keras.initializers.Zeros
        pqc = tfq.layers.PQC(self.make_circuit(self.qubits), readout_operators, differentiator=diff, initializer=init)(inputs)
        model = tf.keras.Model(inputs=inputs, outputs=pqc)
        return model

    def convert_data(self, classical_data, flag=True):
        ops = cirq.Circuit()
        max_scale = max(classical_data)
        min_scale = min(classical_data)
        for i, ang in enumerate(classical_data):
            ang = 2*np.pi*(ang-min_scale)/(max_scale-min_scale)
            ops.append(cirq.rx(np.pi * ang).on(self.qubits[i]))
            ops.append(cirq.rz(np.pi * ang).on(self.qubits[i]))
        if flag:
            return tfq.convert_to_tensor([ops])
        else:
            return ops

    def one_qubit_unitary(self, bit, symbols):
        return cirq.Circuit(
            cirq.X(bit)**symbols[0],
            cirq.Y(bit)**symbols[1],
            cirq.Z(bit)**symbols[2])

    def two_qubit_pool(self, source_qubit, sink_qubit, symbols):
        pool_circuit = cirq.Circuit()
        sink_basis_selector = self.one_qubit_unitary(sink_qubit, symbols[0:3])
        source_basis_selector = self.one_qubit_unitary(source_qubit, symbols[3:6])
        pool_circuit.append(sink_basis_selector)
        pool_circuit.append(source_basis_selector)
        pool_circuit.append(cirq.CNOT(control=source_qubit, target=sink_qubit))
        pool_circuit.append(sink_basis_selector**-1)
        return pool_circuit

    def make_circuit(self, qubits):
        m = cirq.Circuit()
        no_vars = self.no_qubits*3*3 + 2*6
        
        no_vars_str = "q0:"+str(no_vars)
        symbols = sympy.symbols(no_vars_str) # n qubits * 3 weights per bit * 3 layers + 2 * 6 pooling = 36 + 12 = 48
        m += self.layer(symbols[:3*self.no_qubits], qubits)
        m += self.layer(symbols[3*self.no_qubits:2*3*self.no_qubits], qubits)
        m += self.layer(symbols[2*3*self.no_qubits:3*3*self.no_qubits], qubits)
        m += self.two_qubit_pool(self.qubits[0], self.qubits[2], symbols[3*3*self.no_qubits:3*3*self.no_qubits+6])
        m += self.two_qubit_pool(self.qubits[1], self.qubits[3], symbols[3*3*self.no_qubits+6:])
        return m
    
    def layer(self, weights, qubits):
        l = cirq.Circuit()

        for i in range(len(qubits) - 1):
            l.append(cirq.CNOT(qubits[i], qubits[i+1]))
        l.append([cirq.Moment([cirq.rx(weights[j]).on(qubits[j]) for j in range(self.no_qubits)])])
        l.append([cirq.Moment([cirq.ry(weights[j + self.no_qubits]).on(qubits[j]) for j in range(self.no_qubits)])])
        l.append([cirq.Moment([cirq.rz(weights[j + 2*self.no_qubits]).on(qubits[j]) for j in range(self.no_qubits)])])
        return l
    
    def remember(self, state, action, reward, next_state, done):
        i = self.counter % self.buff
        self.states[i] = state
        self.actions[i] = action
        self.rewards[i] = reward
        self.next_states[i] = next_state
        self.dones[i] = int(done)
        self.counter += 1

    def get_action(self, obs):
        if random.random() < self.epsilon: 
            return np.random.choice(self.action_space)
        else:
            return np.argmax(self.q_network.predict(self.convert_data(obs)))
    def train(self):
        batch_indices = np.random.choice(min(self.counter, self.buff), self.batch)
        state_batch = tfq.convert_to_tensor([self.convert_data(i, False) for i in self.states[batch_indices]])
        action_batch = tf.convert_to_tensor(self.actions[batch_indices])
        action_batch = [[i, action_batch[i][0]] for i in range(len(action_batch))]
        reward_batch = tf.convert_to_tensor(self.rewards[batch_indices])
        dones_batch = tf.convert_to_tensor(self.dones[batch_indices])
        reward_batch = tf.cast(reward_batch, dtype=tf.float32)
        action_batch = tf.cast(action_batch, dtype=tf.int32)
        dones_batch = tf.cast(dones_batch, dtype=tf.float32)
        next_state_batch = tfq.convert_to_tensor([self.convert_data(i, False) for i in self.next_states[batch_indices]])

        with tf.GradientTape() as tape:
            next_q = self.q_network(next_state_batch)
            y = reward_batch + (1 - dones_batch) * self.gamma * next_q
            q_guess = self.q_network(state_batch, training=True)
            pred = tf.gather_nd(q_guess, action_batch)
            pred = tf.reshape(pred, [self.batch, 1])
            msbe = tf.math.reduce_mean(tf.math.square(y - pred))
            self.msbe=msbe
        grads = tape.gradient(msbe, self.q_network.trainable_variables)
        self.opt.apply_gradients(zip(grads, self.q_network.trainable_variables))
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

ITERATIONS = 750
batch_size = 32
windows = 50
learn_delay = 500
qubits = [4,8,12]

for q in qubits:
  losses = []
  cur_loss = 1
  env = gym.make("Blackjack-v0")
  env.seed(100)
  agent = QDQN_alt(env.action_space.n, 3, q)
  master_path = make_path("/content/drive/MyDrive/quantum_research/black_jack/quantum_models/", "200+"+agent.model_name)
  rewards = []
  losses = []
  cur_loss = 1
  avg_reward = deque(maxlen=ITERATIONS)
  best_avg_reward = -math.inf
  rs = deque(maxlen=windows)
  epi_times = []
  start_time = time.process_time()

  for i in range(ITERATIONS):
      s1 = env.reset()
      total_reward = 0
      episode_losses=[]
      done = False
      episode_start = time.process_time()
      while not done:
          action = agent.get_action(s1)
          s2, reward, done, info = env.step(action)
          total_reward += reward
          agent.remember(s1, action, reward, s2, done)
          if agent.counter > learn_delay and done:
              agent.train()
              episode_losses.append(agent.msbe)
          if done:
              rewards.append(total_reward)
              rs.append(total_reward)
          s1 = s2
      avg = np.mean(rs)
      avg_reward.append(avg)
      if avg > best_avg_reward:
          best_avg_reward = avg
      if len(episode_losses)>0:
          EPISODE_LOSSES= np.asarray(episode_losses)
          AVERAGE_EPISODE_LOSS = np.mean(EPISODE_LOSSES)
          losses.append(AVERAGE_EPISODE_LOSS)
          cur_loss = AVERAGE_EPISODE_LOSS
      else:
          losses.append(cur_loss)
      epi_end = time.process_time() -episode_start
      epi_times.append(epi_end)
      print("\rEpisode {}/{} || Best average reward {}, Current Iteration Reward {}".format(i, ITERATIONS, best_avg_reward, total_reward))
  reward_file = "{h}/rewards".format(h = master_path)
  average_file = "{h}/averages".format(h=master_path)
  times_file = "{h}/times".format(h=master_path)
  loss_file = "{h}/loss".format(h=master_path)
  np.save(reward_file , np.asarray(rewards))
  np.save(average_file , np.asarray(avg_reward))
  np.save(times_file , np.asarray(epi_times))
  np.save(loss_file , np.asarray(losses))

Checking if /content/drive/MyDrive/quantum_research/black_jack/quantum_models/200+QDQN-2021-04-26_qbits4_ADAM_lr0.01_bs32_g0.95_eps1.0_epsmin0.01_epsd0.9 exists...
finished!
/content/drive/MyDrive/quantum_research/black_jack/quantum_models/200+QDQN-2021-04-26_qbits4_ADAM_lr0.01_bs32_g0.95_eps1.0_epsmin0.01_epsd0.9
Episode 0/750 || Best average reward -1.0, Current Iteration Reward -1.0
Episode 1/750 || Best average reward -1.0, Current Iteration Reward -1.0
Episode 2/750 || Best average reward -0.6666666666666666, Current Iteration Reward 0.0
Episode 3/750 || Best average reward -0.25, Current Iteration Reward 1.0
Episode 4/750 || Best average reward -0.25, Current Iteration Reward -1.0
Episode 5/750 || Best average reward -0.16666666666666666, Current Iteration Reward 1.0
Episode 6/750 || Best average reward -0.16666666666666666, Current Iteration Reward -1.0
Episode 7/750 || Best average reward -0.16666666666666666, Current Iteration Reward 0.0
Episode 8/750 || Best average reward -0